# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Female\GeneCpG\Gastro
df = pd.read_csv('GastroCateg_Methylome_Female.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\Only_Female\GeneCpG\Gastro
(150, 36563)


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,SCOPA_Gastro_Category,cg03441945,cg21094410,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.892410,0.015756,...,0.874283,0.886970,0.903544,0.918856,0.869453,0.889527,0.853304,0.862654,0.929857,0.889773
1,200991620021_R03C01,3002,2,1,2,67.6,4,1,0.922532,0.013309,...,0.843815,0.861038,0.870442,0.896617,0.857573,0.832123,0.807071,0.846446,0.914551,0.874165
2,200989060105_R03C01,3003,2,1,2,56.7,3,0,0.876056,0.011801,...,0.857097,0.881147,0.912999,0.897845,0.836257,0.882631,0.835665,0.896638,0.944095,0.914684


In [3]:
df1 = df.drop(['Sentrix','PATNO','Gender','HYS','Age','AgeCate','SCOPA_Gastro_Category'], axis=1)
df1.head(2)

,APPRDX,cg03441945,cg21094410,cg22926093,cg25025609,cg16704802,cg06228737,cg00220102,cg19016494,cg12247200,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,2,0.892410,0.015756,0.011101,0.018491,0.020476,0.962482,0.677222,0.029560,0.929821,...,0.874283,0.886970,0.903544,0.918856,0.869453,0.889527,0.853304,0.862654,0.929857,0.889773
1,1,0.922532,0.013309,0.012393,0.018230,0.018660,0.924050,0.577240,0.019605,0.937446,...,0.843815,0.861038,0.870442,0.896617,0.857573,0.832123,0.807071,0.846446,0.914551,0.874165


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [5]:
print(dframe1.head(2))
dframe1.to_csv ("Gastro_AllCpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.528571  0.140698
1  0.542857  0.134012


## NMI

In [6]:
df = pd.read_csv('NMI_Top50_Gastro_Methylome_Female.csv')
print(df.shape)
df.head(3)

(150, 52)


,PATNO,APPRDX,cg20699097,cg12162792,cg14268784,cg10615711,cg18248290,cg21028211,cg22136726,cg19138499,...,cg13337307,cg04162413,cg17255878,cg14319793,cg16694455,cg25503540,cg00877661,cg27493785,cg02665154,cg02450124
0,3000,2,0.561816,0.855302,0.703388,0.673214,0.898221,0.038356,0.736337,0.920851,...,0.901061,0.852051,0.019036,0.882989,0.927031,0.785661,0.886459,0.446027,0.038008,0.929601
1,3002,1,0.522515,0.887998,0.752473,0.820693,0.913113,0.025499,0.699776,0.928081,...,0.876021,0.886179,0.034370,0.903554,0.907496,0.801061,0.861123,0.716900,0.035614,0.940119
2,3003,1,0.504891,0.881857,0.742175,0.840767,0.909916,0.033003,0.667122,0.926965,...,0.920221,0.870132,0.015661,0.891902,0.908508,0.847050,0.854343,0.690082,0.038067,0.919557


In [7]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg20699097,cg12162792,cg14268784,cg10615711,cg18248290,cg21028211,cg22136726,cg19138499,cg00332249,...,cg13337307,cg04162413,cg17255878,cg14319793,cg16694455,cg25503540,cg00877661,cg27493785,cg02665154,cg02450124
0,2,0.561816,0.855302,0.703388,0.673214,0.898221,0.038356,0.736337,0.920851,0.070986,...,0.901061,0.852051,0.019036,0.882989,0.927031,0.785661,0.886459,0.446027,0.038008,0.929601
1,1,0.522515,0.887998,0.752473,0.820693,0.913113,0.025499,0.699776,0.928081,0.082550,...,0.876021,0.886179,0.034370,0.903554,0.907496,0.801061,0.861123,0.716900,0.035614,0.940119


In [8]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
print(dframe1.head(2))
dframe1.to_csv ("Gastro_NMICpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.777143  0.148901
1  0.725714  0.139503


## Boruta

In [10]:
df = pd.read_csv('Boruta_Top50_Gastro_Methylome_Female.csv')
print(df.shape)
df.head(3)

(150, 52)


,PATNO,APPRDX,cg06407053,cg18248290,cg26950867,cg25909697,cg15995909,cg00882235,cg01574564,cg05896001,...,cg11278080,cg16980143,cg07924503,cg26835636,cg18086475,cg26096837,cg10726394,cg15713391,cg02091207,cg24637974
0,3000,2,0.113583,0.898221,0.780498,0.727607,0.924530,0.058645,0.045334,0.795810,...,0.627060,0.834242,0.289985,0.019120,0.768044,0.144189,0.022090,0.927416,0.928819,0.914291
1,3002,1,0.106448,0.913113,0.756753,0.665377,0.912166,0.070943,0.030066,0.857006,...,0.616989,0.661617,0.300905,0.012876,0.688672,0.147451,0.010402,0.918400,0.875282,0.877101
2,3003,1,0.094395,0.909916,0.657526,0.595578,0.923107,0.052917,0.029957,0.788452,...,0.604616,0.835319,0.221742,0.018720,0.783019,0.235125,0.009224,0.937921,0.877028,0.860395


In [11]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg06407053,cg18248290,cg26950867,cg25909697,cg15995909,cg00882235,cg01574564,cg05896001,cg23446297,...,cg11278080,cg16980143,cg07924503,cg26835636,cg18086475,cg26096837,cg10726394,cg15713391,cg02091207,cg24637974
0,2,0.113583,0.898221,0.780498,0.727607,0.924530,0.058645,0.045334,0.795810,0.052424,...,0.627060,0.834242,0.289985,0.019120,0.768044,0.144189,0.022090,0.927416,0.928819,0.914291
1,1,0.106448,0.913113,0.756753,0.665377,0.912166,0.070943,0.030066,0.857006,0.053269,...,0.616989,0.661617,0.300905,0.012876,0.688672,0.147451,0.010402,0.918400,0.875282,0.877101


In [12]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
print(dframe1.head(2))
dframe1.to_csv ("Gastro_BorutaCpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.768571  0.150889
1  0.822857  0.152690


## BorutaR

In [14]:
df = pd.read_csv('BorutaR_Gastro_CpG_Female.csv')
print(df.shape)
df.head(3)

(150, 14)


,PATNO,APPRDX,cg20545648,cg24573261,cg04670359,cg08596655,cg15727409,cg26085473,cg26950867,cg23446297,cg12347084,cg12103037,cg01933308,cg05896001
0,3000,2,0.741250,0.366270,0.049400,0.026157,0.271338,0.009988,0.780498,0.052424,0.025702,0.076198,0.115829,0.795810
1,3002,1,0.656437,0.667207,0.029238,0.017470,0.352620,0.015091,0.756753,0.053269,0.018231,0.088004,0.040682,0.857006
2,3003,1,0.741720,0.621821,0.037729,0.039910,0.309989,0.011999,0.657526,0.079942,0.015661,0.079265,0.078564,0.788452


In [15]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg20545648,cg24573261,cg04670359,cg08596655,cg15727409,cg26085473,cg26950867,cg23446297,cg12347084,cg12103037,cg01933308,cg05896001
0,2,0.741250,0.366270,0.049400,0.026157,0.271338,0.009988,0.780498,0.052424,0.025702,0.076198,0.115829,0.795810
1,1,0.656437,0.667207,0.029238,0.017470,0.352620,0.015091,0.756753,0.053269,0.018231,0.088004,0.040682,0.857006


In [16]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [17]:
print(dframe1.head(2))
dframe1.to_csv ("Gastro_BorutaRCpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.728571  0.176705
1  0.854286  0.126136


## Common

In [18]:
df = pd.read_csv('Common_Gastro_CpG_Female.csv')
print(df.shape)
df.head(3)

(150, 13)


,PATNO,APPRDX,cg03181829,cg05896001,cg08596655,cg12103037,cg15713391,cg15727409,cg18248290,cg19138499,cg20545648,cg23446297,cg26950867
0,3000,2,0.039903,0.795810,0.026157,0.076198,0.927416,0.271338,0.898221,0.920851,0.741250,0.052424,0.780498
1,3002,1,0.058078,0.857006,0.017470,0.088004,0.918400,0.352620,0.913113,0.928081,0.656437,0.053269,0.756753
2,3003,1,0.035479,0.788452,0.039910,0.079265,0.937921,0.309989,0.909916,0.926965,0.741720,0.079942,0.657526


In [19]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg03181829,cg05896001,cg08596655,cg12103037,cg15713391,cg15727409,cg18248290,cg19138499,cg20545648,cg23446297,cg26950867
0,2,0.039903,0.795810,0.026157,0.076198,0.927416,0.271338,0.898221,0.920851,0.741250,0.052424,0.780498
1,1,0.058078,0.857006,0.017470,0.088004,0.918400,0.352620,0.913113,0.928081,0.656437,0.053269,0.756753


In [20]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [21]:
print(dframe1.head(2))
dframe1.to_csv ("Gastro_CommonCpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0      0.78  0.134650
1      0.74  0.163358


## Random 50

In [22]:
df = pd.read_csv('GastroCateg_Methylome_Female.csv')
print(df.shape)
df.head(3)

(150, 36563)


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,SCOPA_Gastro_Category,cg03441945,cg21094410,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.892410,0.015756,...,0.874283,0.886970,0.903544,0.918856,0.869453,0.889527,0.853304,0.862654,0.929857,0.889773
1,200991620021_R03C01,3002,2,1,2,67.6,4,1,0.922532,0.013309,...,0.843815,0.861038,0.870442,0.896617,0.857573,0.832123,0.807071,0.846446,0.914551,0.874165
2,200989060105_R03C01,3003,2,1,2,56.7,3,0,0.876056,0.011801,...,0.857097,0.881147,0.912999,0.897845,0.836257,0.882631,0.835665,0.896638,0.944095,0.914684


In [23]:
df1 = df.drop(['Sentrix','PATNO','Gender','HYS','Age','AgeCate','SCOPA_Gastro_Category'], axis=1)
df1.head(2)

,APPRDX,cg03441945,cg21094410,cg22926093,cg25025609,cg16704802,cg06228737,cg00220102,cg19016494,cg12247200,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,2,0.892410,0.015756,0.011101,0.018491,0.020476,0.962482,0.677222,0.029560,0.929821,...,0.874283,0.886970,0.903544,0.918856,0.869453,0.889527,0.853304,0.862654,0.929857,0.889773
1,1,0.922532,0.013309,0.012393,0.018230,0.018660,0.924050,0.577240,0.019605,0.937446,...,0.843815,0.861038,0.870442,0.896617,0.857573,0.832123,0.807071,0.846446,0.914551,0.874165


In [24]:
df2 = df1.drop (['APPRDX'], axis=1)

In [25]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [26]:
print(dframe1.head(2))
dframe1.to_csv ("Gastro_Rand50CpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.500000  0.183503
1  0.485714  0.145686


## Random 12

In [27]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        df3 = df2.sample(12, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [28]:
print(dframe1.head(2))
dframe1.to_csv ("Gastro_Rand12CpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.385714  0.143570
1  0.500000  0.194307
